In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import img_to_array
from keras_preprocessing.image import load_img
from keras.optimizers import RMSprop,SGD,Adam
from sklearn.svm import SVR 
import argparse
from imutils import paths
import os

import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train = '/content/drive/Shareddrives/Graduation Project/Dataset/Teeth Images Dataset'

In [ ]:
count = 0
dir_path = r'/content/drive/Shareddrives/Graduation Project/Dataset/Teeth Images Dataset/Sharks'
for path in os.scandir(dir_path):
    if path.is_file():
        count += 1
print('images count:', count)

images count: 103


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(180, 180),
    batch_size=30,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train, # same directory as training data
    target_size=(180, 180),
    batch_size=30,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 591 images belonging to 5 classes.
Found 145 images belonging to 5 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(5,activation = "softmax")   #Adding the Output Layer
])

In [ ]:
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [ ]:
model.fit(x=train_generator,steps_per_epoch=len(train_generator),
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    # steps_per_epoch=150 // bs,
                    epochs=10,
                    verbose=1
                    )

Epoch 1/10
20/20 [==============================] - 273s 14s/step - loss: 0.7832 - acc: 0.7174 - val_loss: 0.7136 - val_acc: 0.7241
Epoch 2/10
20/20 [==============================] - 196s 10s/step - loss: 0.7057 - acc: 0.7428 - val_loss: 0.7269 - val_acc: 0.7862
Epoch 3/10
20/20 [==============================] - 198s 10s/step - loss: 0.6838 - acc: 0.7699 - val_loss: 0.7266 - val_acc: 0.8207
Epoch 4/10
20/20 [==============================] - 196s 10s/step - loss: 0.6023 - acc: 0.8037 - val_loss: 0.6898 - val_acc: 0.7931
Epoch 5/10
20/20 [==============================] - 193s 10s/step - loss: 0.5374 - acc: 0.8190 - val_loss: 0.7354 - val_acc: 0.8069
Epoch 6/10
20/20 [==============================] - 199s 10s/step - loss: 0.5161 - acc: 0.8139 - val_loss: 0.7308 - val_acc: 0.8207
Epoch 7/10
20/20 [==============================] - 196s 10s/step - loss: 0.4622 - acc: 0.8562 - val_loss: 0.6315 - val_acc: 0.8621
Epoch 8/10
20/20 [==============================] - 195s 10s/step - loss: 0.

In [ ]:
model.evaluate(validation_generator)

5/5 [==============================] - 46s 9s/step - loss: 0.5507 - acc: 0.8621


[0.5507011413574219, 0.8620689511299133]

In [ ]:
pred = model.predict(validation_generator)

5/5 [==============================] - 36s 7s/step


In [ ]:
classes = [np.argmax(element) for element in pred]
print('Predicted values:', classes[:10])
# print('Actual values:', validation_generator[:10])

Predicted values: [1, 4, 1, 0, 4, 4, 1, 1, 1, 4]
